Neste hackathon, o desafio do seu time é desenvolver uma solução utilizando o conjunto
de dados chamado "Diamonds". Problemática: “Sou dono de uma loja de diamantes e
relógios, chamada Diamonds; somos a maior loja de diamantes do Brasil. Diamonds
possui dados sobre os preços e outros atributos de quase 54 mil diamantes. Contudo, nós
não sabemos o que fazer com esses dados ou como analisá-los. Por isso, estamos
buscando uma equipe de especialistas qualificados que consigam desenvolver uma
solução utilizando nossos dados.” Seu desafio é desenvolver uma análise, aplicação,
ideia, o que for, utilizando este conjunto de dados e, é claro, Aprendizado de Máquina!

In [ ]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) # mostrando o path do csv

SEED = 42

In [ ]:
# carregando os dados do csv
df = pd.read_csv("/kaggle/input/diamondscsv/diamonds.csv")
df.head(8) # mostrando as 8 primeiras linhas do csv

Especificações de um diamante:

* **Quilate** - peso em quilates do diamante.
* **Corte** - Descreve a qualidade do corte do diamante (do melhor ao pior: Ideal, Premium, Muito Bom, Bom e Regular).
* **Cor** - Cor do diamante (do melhor para o pior: D, E, F, G, H, I e J).
* **Clareza** - Uma medida de quão claro é o diamante (do melhor ao pior: IF, VVS1, VVS2, VS1, VS2, SI1, SI2 e I1).
* **Profundidade** - A altura de um diamante, medida da culaça à mesa, dividida pelo diâmetro médio do rondiz (%).
* **Mesa** - A largura de uma mesa de diamante expressa como uma porcentagem do diâmetro médio (%).
* **x** - Comprimento do diamante (mm).
* **y** - Largura do diamante (mm).
* **z** - Profundidade do diamante (mm).
* **Preço** - Preço do diamante.

In [ ]:
# Obtendo algumas informações sobre os dados
df.info()

In [ ]:
# Verificando de existe valores ausentes
if df.isnull().sum().any() == False:
    print("Não há valores ausentes")
else:
    print("Há algo errado aqui")

# <font size="4">Verificando linhas duplicadas</font>

Vamos verificar se há linhas duplicadas, mas primeiro precisamos excluir a coluna "Sem nome: 0", pois ela é inútil para nós e tem um valor diferente para cada linha, ou seja, se não descartarmos esta coluna, o número de linhas duplicadas sempre serão zero.

In [ ]:
# Descartando a coluna "Sem nome: 0"
df = df.drop(["Unnamed: 0"], axis = 1)

# Verificando linhas duplicadas
print("Quantidade de linhas duplicadas: ", df.duplicated().sum())

In [ ]:
# Apagando as linhas duplicadas
df = df.drop_duplicates().reset_index(drop = True)

# <font size="4">Descrevendo os dados</font>

In [ ]:
# Dados numéricos
df.describe().T

In [ ]:
# Dados categóricos
df.describe(include = "O").T

Depois de usar df.describe (). T, podemos ver que os valores mínimos de x, y e z são iguais a zero. Vamos dar uma olhada neles.

In [ ]:
format_dict = {"carat" : "{:.2f}", "depth" : "{:.1f}", "table" : "{:.1f}", "x" : "{:.2f}", "y" : "{:.2f}", "z" : "{:.2f}"}
df_zero = df.loc[(df["x"] == 0) | (df["y"] == 0) | (df["z"] == 0)]
df_zero.style.apply(lambda x: ["background: yellow" if n == 0 else "" for n in x], axis = 1).format(format_dict)

Sabemos que esses valores devem ser maiores que zero, mas por algum motivo não são. Portanto, vamos tratá-los como valores ausentes.

In [ ]:
# Transformando-os em valores ausentes
df.loc[df["x"] == 0, "x"] = np.nan 
df.loc[df["y"] == 0, "y"] = np.nan 
df.loc[df["z"] == 0, "z"] = np.nan 

# Vendo o número dos novos valores ausentes
df[["x", "y", "z"]].isnull().sum()

Agora, temos três colunas com valores ausentes. Vamos ver as correlações mais fortes para cada um deles

In [ ]:
def get_corr(col):
    return df.corr().unstack()[col].sort_values(ascending = False)

In [ ]:
print("x correlações\n\n{0}\n\n{3}\n\ny correlações\n\n{1}\n\n{3}\n\nz correlações\n\n{2}".format(get_corr("x"), get_corr("y"), get_corr("z"), 25*"-"))

* **x** a correlação mais forte é com o **quilate**
* **y** a correlação mais forte é com **x**
* **z** a correlação mais forte é com **x**

Como **quilate** não tem valores ausentes e tem uma forte correlação com **x**, **y** e **z**, vamos supor que a correlação mais forte de todas as dimensões é com **quilate** , porque em alguns casos mais de um valor de dimensão está errando na mesma linha e vamos usar a mediana dessa correlação para preenchê-los.

In [ ]:
def fill_nan_values(col):
    carat = df.groupby(["carat"])[col].median()
    index_list = list(df.loc[df[col].isnull() == True].sort_values(by = "carat", ascending = False).index)
    for i in index_list:
        carat_value = df.loc[i, "carat"]
        new_value = carat[carat_value]
        df.loc[i, col] = new_value
        print("Quilate: {0} / Mediana {1} Valor: {2}".format(carat_value, col, new_value))
    return df.iloc[index_list].style.applymap(lambda x: "background-color: limegreen", subset = col).format(format_dict)


Coloquei os valores em ordem decrescente, para que possamos visualizar melhor essa correlação.

In [ ]:
# Corrigindo a coluna "X"
fill_nan_values("x")

In [ ]:
# Corrigindo a coluna "Y"
fill_nan_values("y")

In [ ]:
# Corrigindo a coluna "Z"
fill_nan_values("z")

# <font size="4">Outliers</font>

Existem muitos métodos para descobrir valores discrepantes, mas aqui vamos usar o mais simples: visualizá-los manualmente. Para isso, precisamos traçar gráficos mostrando a relação entre todos os recursos numéricos e o alvo (preço).

In [ ]:
def highlight_outliers(outliers, col):
    outliers_index = outliers.index
    i = pd.IndexSlice[outliers_index, col]
    return outliers.style.applymap(lambda x: "background-color: red", subset = i).format(format_dict)

In [ ]:
sns.set_style("whitegrid")
c = "springgreen"

plt.figure(figsize = (12, 18))
plt.subplot(3, 2, 1)
plt.title("Preço X Quilate")
sns.regplot(data = df, x = "price", y = "carat", color = c, line_kws = {"color" : "black"})
plt.subplot(3, 2, 2)
plt.title("Preço X Profundidade")
sns.regplot(data = df, x = "price", y = "depth", color = c, line_kws = {"color" : "black"})
plt.subplot(3, 2, 3)
plt.title("Preço X Mesa")
sns.regplot(data = df, x = "price", y = "table", color = c, line_kws = {"color" : "black"})
plt.subplot(3, 2, 4)
plt.title("Preço X x")
sns.regplot(data = df, x = "price", y = "x", color = c, line_kws = {"color" : "black"})
plt.subplot(3, 2, 5)
plt.title("Preço X y")
sns.regplot(data = df, x = "price", y = "y", color = c, line_kws = {"color" : "black"})
plt.subplot(3, 2, 6)
plt.title("Preço X z")
sns.regplot(data = df, x = "price", y = "z", color = c, line_kws = {"color" : "black"})
plt.show()

Olhando para estes gráficos, podemos ver três valores que estão realmente distantes dos outros (observe a diferença de espessura entre os dois últimos gráficos e os outros). Vamos dar uma olhada neles.

* <font size="3">Preço x Y</font>

In [ ]:
df_outliers = df.loc[df["y"] > 30].copy()
highlight_outliers(df_outliers, "y")

* <font size="3">Preço x Z</font>

In [ ]:
df_outliers = df.loc[df["z"] > 30].copy()
highlight_outliers(df_outliers, "z")

Esses valores são verdadeiros outliers, vamos tratá-los como valores ausentes e fazer a mesma coisa que fizemos antes.

In [ ]:
# Transformando-os em valores nulos
df.loc[df["y"] > 30, "y"] = np.nan
df.loc[df["z"] > 30, "z"] = np.nan

In [ ]:
# Corrigindo a coluna Y
fill_nan_values("y")

In [ ]:
# Corrigindo a coluna Z
fill_nan_values("z")

Agora que estamos livres desses valores discrepantes verdadeiros, vamos analisar os possíveis valores discrepantes.

* <font size="3">Preço X Profundidade</font>

In [ ]:
df_outliers = df.loc[(df["depth"] > 75) | (df["depth"] < 45)].copy()
highlight_outliers(df_outliers, "depth")

Bem, mesmo que esses valores sejam consideravelmente distantes de outros, eles não são valores absurdos (por exemplo, 550%). Vamos deixá-los como estão.

* <font size="3">Preço X Mesa</font>

In [ ]:
df_outliers = df.loc[(df["table"] > 90) | (df["depth"] < 45)].copy()
highlight_outliers(df_outliers, "table")

A mesma coisa de antes.

* <font size="3">Preço x Y</font>

In [ ]:
df_outliers = df.loc[df["z"] < 2].copy()
highlight_outliers(df_outliers, "z")


Por alguma razão, os valores de z são iguais ao quilate.

In [ ]:
df.loc[df["carat"] == df["z"], ["carat", "z"]]

Isso não pode ser apenas uma coincidência, pois apenas este três tem esses valores iguais (carat = z), então vamos corrigi-los, repetindo o processo.

In [ ]:
# Transformando-os em valores nulos
df.loc[df["z"] < 2, "z"] = np.nan

In [ ]:
fill_nan_values("z")

# <font size="4">Visualização dos dados</font>

In [ ]:
cut_palette = ["darkturquoise", "lightskyblue", "paleturquoise", "lightcyan", "azure"]
color_palette = ["cadetblue", "deepskyblue", "darkturquoise", "lightskyblue", "paleturquoise", "lightcyan", "azure"]
clarity_palette = ["cadetblue", "deepskyblue", "darkturquoise", "lightskyblue", "paleturquoise", "lightcyan", "azure", "ghostwhite"]

df["cut"] = pd.Categorical(df["cut"], categories = ["Ideal", "Premium", "Very Good", "Good", "Fair"], ordered = True)
df["color"] = pd.Categorical(df["color"], categories = ["D", "E", "F", "G", "H", "I", "J"], ordered = True)
df["clarity"] = pd.Categorical(df["clarity"], categories = ["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1"], ordered = True)

In [ ]:
df_cut = df["cut"].value_counts()

plt.figure(figsize = (7,7))
plt.pie(data = df_cut, x = df_cut.values, labels = df_cut.index, autopct = "%.2f%%", pctdistance = 0.8, colors = cut_palette )
circle = plt.Circle(xy = (0, 0), radius = 0.5, facecolor = 'white')
plt.gca().add_artist(circle)
plt.title("% da qualidade de corte de diamante", size = 16)
plt.show()

Lembrando o pedido: Ideal> Premium> Muito Bom> Bom> Razoável

In [ ]:
position = 0
for cut in df_cut:
    print("{0} quality cuts: {1}".format(df_cut.index[position], df_cut.values[position]))
    position += 1

Como podemos ver, existem muito mais diamantes lapidados de alta qualidade. Agora vamos ver como isso afeta o preço.

In [ ]:
plt.figure(figsize = (9, 6))
sns.barplot(data = df, x = "cut", y = "price", color = c)
plt.title("Relação entre corte e preço", size = 16)
plt.show()

Bem... isso foi inesperado, por que o preço médio dos diamantes com cortes ideais são mais baixos do que todos os outros diamantes com corte de qualidade inferior?

Vejamos qual recurso tem maior impacto no preço.

In [ ]:
get_corr("price")

É o quilate!

Então, se o quilate é o mais importante, quando estamos procurando os diamantes com cortes ideais, deveriam ter valor do quilate menor.

In [ ]:
df.groupby(["cut"])["carat"].mean()

Aqui está, é por isso que o preço médio dos diamantes de lapidação ideal é o mais baixo.

In [ ]:
df_color = df["color"].value_counts()

plt.figure(figsize = (7,7))
plt.pie(data = df_color, x = df_color.values, labels = df_color.index, autopct = "%.2f%%", pctdistance = 0.8, startangle = 40, colors = color_palette)
circle = plt.Circle(xy = (0, 0), radius = 0.5, facecolor = 'white')
plt.gca().add_artist(circle)
plt.title("% pela cor do diamante", size = 16)
plt.show()

Lembrando a ordem: D> E> F> G> H> I> J

In [ ]:
position = 0
for color in df_color:
    print("{0} color diamonds: {1}".format(df_color.index[position], df_color.values[position]))
    position += 1

In [ ]:
plt.figure(figsize = (9, 6))
sns.barplot(data = df, x = "color", y = "price", color = c)
plt.title("Relação entre a cor do diamante e o preço", size = 16)
plt.show()

Novamente, o preço médio dos diamantes com cores melhores é menor do que todos os outros diamantes com cores piores.

In [ ]:
df.groupby(["color"])["carat"].mean()

E o motivo é o mesmo do corte.

In [ ]:
df_clarity = df["clarity"].value_counts()

plt.figure(figsize = (7,7))
plt.pie(data = df_clarity, x = df_clarity.values, labels = df_clarity.index, autopct = "%.2f%%", pctdistance = 0.8, colors = clarity_palette)
circle = plt.Circle(xy = (0, 0), radius = 0.5, facecolor = 'white')
plt.gca().add_artist(circle)
plt.title("% pela clareza do diamante", size = 16)
plt.show()

Lembrando a ordem: IF> VVS1> VVS2> VS1> VS2> SI1> SI2> I1

In [ ]:
position = 0
for color in df_clarity:
    print("{0} clarity diamonds: {1}".format(df_clarity.index[position], df_clarity.values[position]))
    position += 1

In [ ]:
plt.figure(figsize = (9, 6))
sns.barplot(data = df, x = "clarity", y = "price", color = c)
plt.title("Relação entre a clareza do diamante e o Preço", size = 16)
plt.show()

Novamente...

In [ ]:
df.groupby(["clarity"])["carat"].mean()

E o motivo é o mesmo.

# <font size="4">Data Preparation</font>

In [ ]:
# Dividindo os dados em recursos e destino
X = df.drop(["price"], axis = 1).copy()
y = df["price"].copy()

Alguns algoritmos respondem melhor com variáveis fictícias, outros com codificador de rótulo ordenado corretamente, então vamos criar dois diferentes.

In [ ]:
# Variáveis fictícias
X_dummies = X.copy()
X_dummies = pd.get_dummies(data = X_dummies, columns = ["clarity", "color", "cut"], prefix = ["clarity", "color", "cut"], drop_first = True).copy()

# Dimensionando os dados
ss = StandardScaler()
X_dummies = ss.fit_transform(X_dummies)

# Codificador de etiqueta
X_encoder = X.copy()
X_encoder["cut"] = X_encoder["cut"].replace({"Ideal": 1, "Premium": 2, "Very Good": 3, "Good": 4, "Fair": 5}).copy()
X_encoder["color"] = X_encoder["color"].replace({"D": 7, "E": 6, "F": 5, "G": 4, "H": 3, "I": 2, "J": 1}).copy()
X_encoder["clarity"] = X_encoder["clarity"].replace({"IF" : 8, "VVS1" : 7, "VVS2" : 6, "VS1" : 5, "VS2" : 4, "SI1" : 3, "SI2" : 2, "I1" : 1}).copy()

In [ ]:
# K-folds para validação cruzada
kf = KFold(n_splits = 10, random_state = SEED, shuffle = True)

# Avaliando a validação cruzada
scoring = ["r2", "neg_mean_absolute_error", "neg_mean_squared_error"]

In [ ]:
model_scores = {"train" : [],
                "test" : [],
                "mae" : [],
                "mse" : [],
                "rmse" : []}

def get_results(clf, features):
    scores = cross_validate(clf, features, y, cv = kf, scoring = scoring, return_train_score = True)
    train_score = scores["train_r2"].mean()
    model_scores["train"].append(train_score)
    test_score = scores["test_r2"].mean()
    model_scores["test"].append(test_score)
    mae = np.absolute(scores["test_neg_mean_absolute_error"]).mean()
    model_scores["mae"].append(mae)
    mse = np.absolute(scores["test_neg_mean_squared_error"]).mean()
    model_scores["mse"].append(mse)
    rmse = np.sqrt(mse)
    model_scores["rmse"].append(rmse)
    print("train score: {0:.4f}\nR2 score: {1:.4f}\nMAE: {2:.2f}\nMSE: {3:.2f}\nRMSE: {4:.2f}".format(train_score, test_score, mae, mse, rmse))

Observe que todos os valores são a média de todas as execuções de validação cruzada.

In [ ]:
lr = LinearRegression()
get_results(lr, X_dummies)

In [ ]:
print("train score\n")
print("Linear Regression: {0:.4f}".format(model_scores["train"][0]))
print("\nR2 score\n")
print("Linear Regression: {0:.4f}".format(model_scores["test"][0]))
print("\nMAE\n")
print("Linear Regression: {0:.2f}".format(model_scores["mae"][0]))
print("\nMSE\n")
print("Linear Regression: {0:.2f}".format(model_scores["mse"][0]))
print("\nRMSE\n")
print("Linear Regression: {0:.2f}".format(model_scores["rmse"][0]))